# Cvičenie 10: Simulácia letu komára

Určite ste už zažili, že ste sa snažili zaspať na konci dlhého letného dňa, no tesne pred tým, že by ste sa ponorili do spánku, ste začuli [iritujúci zvuk komára](https://www.youtube.com/watch?v=AMg-IgW7Dzg). Zvuk je niekedy intenzívnejši, neskôr ho ledva počujete, ako komár letí v izbe na prvý pohľad úplne náhodne. V skutočnosti sa ale nejedná o náhodný let, komár naviguje pomocou čuchu a riadi sa jednoduchými pravidlami.

Na dnešnom cvičení pomocou objektovo orientovaného programovania a simulácie si ukážeme, ako komár dokáže nájsť svoju korisť vďaka geniality prírody. Na konci cvičenia teda budete mať implementáciu kódu, ktorá nasimuluje let komára nasledujúcim spôsobom (modrá bodka reprezentuje človeka, červený krížik komára):

![Simulácia letu komára](sources/lab10/animation.gif)

Simulácia sa skladá z niekoľkých konceptov, a to konkrétne:

* `Field` - dvojrozmerný priestor, v ktorom sa nachádzajú naši aktéri (komár a človek)
* aktéri `Mosquito` a `Human` - reprezentujú entity, ktoré sa pohybujú v priestore; pre zjednotenie funkcionality vytvoríme aj nadtriedu `Actor`
* pre reprezentáciu pohybu použijeme pomocné triedy `Location`, ktorá bude reprezentovať polohu aktéra, a `Direction` pre reprezentáciu pohybu a pre výpočet aktualizovanej polohy.

[Stiahnite si kostru riešenia](sources/lab10/lab10.zip) alebo pracujte v tomto notebooku. Našu implementáciu začneme práve s pomocnými triedami.

## 1. Poloha a pohyb

Pred tým, než zadefinujeme našich aktérov, vytvoríme pomocné triedy pre reprezentáciu ich polohy a pohybu v priestore. Ako prvé sa pozrieme na triedu `Location`, ktorá je už plne implementovaná. Poloha aktéra bude reprezentovaná dvomi súradnicovými hodnotami `x` a `y`. Tie uložíme ako členské premenné v konštruktore. Trieda ďalej obsahuje metódy `move`, ktorá dostane ako parametre zmeny x-ovej a y-ovej súradnicovej hodnoty a vráti novú polohu (objekt typu `Location`) s aktualizovanými hodnotami (pripočíta sa zmena po jednotlivé osi). Metóda `get_coords` vráti x-ové a y-ové súradnicové hodnoty bodu (v zmysle enkapsulácie) a metóda `get_distance` vypočíta vzdialenosť medzi dvomi bodmi v dvojrozmernom priestore pomocou Pytagorovej vety.

In [1]:
from math import sqrt


class Location:
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def move(self, dx, dy):
        return Location(self.x + float(dx), self.y + float(dy))

    def get_coords(self):
        return self.x, self.y

    def get_distance(self, other):
        o_x, o_y = other.get_coords()
        x_dist = self.x - o_x
        y_dist = self.y - o_y
        return sqrt(x_dist ** 2 + y_dist ** 2)

Druhá pomocná trieda sa nazýva `Direction` a nájdete ju v súbore `direction.py`. Táto trieda reprezentuje smer pohybu, a použijeme ju pri výpočte zmeny polohy. Smer pohybu bude definovaný uhlom $\theta$, pričom pre reprezentáciu použijeme radiány.

![Jednotková kružnica](https://www.mathwarehouse.com/unit-circle/images/unit-circle-general-formula-graph.png)

**Úloha:** Opravte konštruktor tak, aby uhol normalizoval na hodnotu v intervale $<0, 2\pi>$. Následne implementujte metódu `move`, ktorá ako parameter dostane vzdialenosť, ktorú daný aktér prekoná jedným krokom (polomer jednotkovej kružnice). Funkcia vracia dvojicu hodnôt, ktorá obsahuje zmenu po x-ovej a po y-ovej súradnici. Použite pritom sínus a kosínus uhla `self.angle`.

In [ ]:
class Direction:
    def __init__(self, angle):
        # angle is given in radians
        # TODO: set angle, and normalize it for range <0, 2pi>
        self.angle = angle

    def move(self, dist):
        # TODO: return difference in x and y values based on angle
        return (-1, -1)

    def get_angle(self):
        return self.angle

## 2. Umiestnenie aktérov

Už máme pripravené definície polohy a pohybu, avšak zatiaľ nevieme, kde sa odohráva simulácia. Práve preto teraz doplníme triedu `Field`, ktorá definuje spôsob narábania s našimi aktérmi. Väčšina funkcionality je už hotová, ostávajú nám iba dve metódy `get_actor_distance` a `found`. Ostatné metódy majú nasledovnú funkcionalitu:

* `move_human`: posunie človeka (podľa definície pohybu v triede `Human`);
* `move_mosquito`: posunie komára (podľa definície pohybu v triede `Mosquito`);
* `get_human_coordinates`: vráti pozíciu človeka;
* `get_mosquito_coordinates`: vráti pozíciu komára;
* `get_human`: vráti objekt človeka;
* `get_mosquito`: vráti objekt komára.

**Úloha:** Chýbajúca metóda `get_actor_distance` má vrátiť vzdialenosť medzi človekom a komárom (použije sa pri definícii pohybu komára) a metóda `found` vráti `boolean` vyjadrujúci, či komár našiel človeka alebo nie. Nájdenie koristi definujeme ako moment, keď vzdialenosť medzi človekom a komárom je menšia ako 2.

In [ ]:
class Field:
    def __init__(self, human, mosquito):
        self.human = human
        self.mosquito = mosquito

        self.human.set_field(self)
        self.mosquito.set_field(self)

    def move_human(self):
        self.human.move()

    def move_mosquito(self):
        self.mosquito.move()

    def get_human_coordinates(self):
        return self.human.get_coords()

    def get_mosquito_coordinates(self):
        return self.mosquito.get_coords()

    def get_human(self):
        return self.human

    def get_mosquito(self):
        return self.mosquito

    def get_actor_distance(self):
        # TODO: return distance between human and mosquito
        return 0

    def found(self):
        # TODO: return if mosquito has found human (distance between them < 2)
        return False


## 3. Postavy v priestore

V priestore simulácie sa nachádzajú dva typy aktérov: komár a človek. Pred tým, než implementujeme prislúchajúce triedy ale vytvoríme abstraktnú nadtriedu `Actor`, ktorá deklaruje funkcionalitu aktérov, a to nasledovne:

* `set_field`: pridávanie aktéra do priestoru;
* `move`: pohyb aktéra, jednotlivé podtriedy ju implementujú;
* `get_coords`: vráti x-ovú a y-ovú súradnicu aktéra;
* `get_location`: vráti pozíciu aktéra ako objekt typu `Location`.

In [ ]:
class Actor:
    def __init__(self, name, speed, x, y):
        self.name = name
        self.speed = speed
        self.loc = Location(x, y)

        self.field = None

    def set_field(self, field):
        self.field = field

    def move(self):
        pass

    def get_coords(self):
        return self.loc.get_coords()

    def get_location(self):
        return self.loc

### 3.1. Človek

Najprv implementujeme triedu `Human`, ktorá bude jednoduchšia. Vzhľadom na to, že väčšina funkcionality je odvodená od nadtriedy, potrebujeme implementovať iba metódu `move`, keďže konštruktor všetko nastaví podľa konštruktora nadtriedy. V metóde `move` implementujte náhodný pohyb človek nasledovným spôsobom:

1. vytvorte objekt typu `Direction` pre reprezentáciu pohybu v náhodnom smere;
2. vypočítajte zmenu súradnicových hodnôt pomocou objektu `Direction`;
3. aktualizujte pozíciu aktéra cez členskú premennú `loc`.

In [ ]:
class Human(Actor):
    def __init__(self, name, speed, x, y):
        super().__init__(name, speed, x, y)

    def move(self):
        # TODO: take a step in a random direction
        pass

### 3.2. Komár

Let komára sa riadi niekoľkými jednoduchými pravidlami, ktoré vyžadujú ukladávanie pomocných hodnôt, pre ktoré vytvoríme členské premenné v konštruktore. Následne vieme implementovať funkcionalitu metódy `move` na základe nasledovných pravidiel:

* Kým komár je vo väčšej vzdialenosti od človeka ako 90, pohybuje sa náhodne.
* Ak vzdialenosť je menšia, ako 90, komár zachytil vôňu krvi človeka. Od tohto momentu bude jeho pohyb riadený.
* Najprv začne letieť v náhodnom smere. Ak po vykonaní kroku je vôňa silnejšia (vzdialenosť je menšia), tak pokračuje v tomto smere. V opačnom prípade si vyberie iný náhodný smer.
* Ak vzdialenosť bude väčšia ako 90, komár stratil vôňu a znova letí náhodným spôsobom.



In [ ]:
class Mosquito(Actor):
    def __init__(self, name, speed, x, y):
        super().__init__(name, speed, x, y)
        self.found_prey = False
        self.last_distance = 0
        self.last_direction = None

    def move(self):
        # TODO: implement mosquito's movement:
        # while distance > 90 - move in a random direction
        # if distance < 90, the mosquito has found the human scent trace
        # fly in random direction, if scent gets stronger, continue
        # if scent gets weaker, turn around
        # if scent is lost, fly in a random direction
        pass


## 4. Simulácia

Ak ste sa dostali do tohto bodu, už máte všetko pripravené k simulácii. Kód simulácie nájdete v súbore `main.py`, a využíva knižnicu `matplotlib` pre vizualizáciu. Ak túto knižnicu ešte nemáte nainštalovanú (je súčasťou Anacondy), môžete ju doinštalovať príkazom `pip install matplotlib` v príkazovom riadku alebo cez grafické rozhranie IDE.

Na začiatku simulácie náhodne vygenerujeme pozíciu človeka, komár začne na pozícii `[10, 10]`. Simulácia trvá až dovtedy, kým komár nenájde človeka, pričom aktualizuje sa iba vykreslený graf, teda pozícia bodov reprezentujúcich človeka (modrý kruh) a komára (červený krížik).

In [ ]:
import random
from time import sleep

import matplotlib.pyplot as plt


human_x = random.randint(-50, 50)
human_y = random.randint(-50, 50)
human = Human("Janko Hrasko", 1, human_x, human_y)
mosquito = Mosquito("Mosquito", 1, 10, 10)

field = Field(human, mosquito)

plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim([-100, 100])
ax.set_ylim([-100, 100])

human_pos = field.get_human_coordinates()
mosquito_pos = field.get_mosquito_coordinates()

human_plot_point, = ax.plot(human_pos, 'bo')
mosquito_plot_point, = ax.plot(mosquito_pos, 'rx')
fig.canvas.draw()
fig.canvas.flush_events()

while not field.found():
    human_pos = field.get_human_coordinates()
    mosquito_pos = field.get_mosquito_coordinates()

    human_plot_point.set_xdata(human_pos[0])
    human_plot_point.set_ydata(human_pos[1])
    mosquito_plot_point.set_xdata(mosquito_pos[0])
    mosquito_plot_point.set_ydata(mosquito_pos[1])

    fig.canvas.draw()
    fig.canvas.flush_events()

    field.move_mosquito()
    sleep(0.1)

Ako môžete vidieť, v simulácii sa zatiaľ pohybuje iba komár, pridajte teda pohyb človeka, náhodne vygenerujte aj začiatočnú pozíciu komára, a všeobecne vyskúšajte rôzne nastavenia. Vaše riešenie môžete tiež rozšíriť o pridávanie prekážok do sveta.